# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn
from azureml.automl.runtime.onnx_convert import OnnxConverter
import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.sklearn import SKLearn

ws = Workspace.get(name="quick-starts-ws-136122")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      )

run = exp.start_logging()

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-capstone'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FC9TAE3S4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_DS3_V2"
compute_name="compute-clusters"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4




### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    print(compute_target.get_status().serialize())

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
run = Run.get_context()
#ws = run.experiment.workspace
dataset = Dataset.get_by_name(ws, name='cancer')
dataset.to_pandas_dataframe()
df=dataset.to_pandas_dataframe()
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.00,0.12,0.28,0.30,0.15,...,25.38,17.33,184.60,2019.00,0.16,0.67,0.71,0.27,0.46,0.12
1,842517,M,20.57,17.77,132.90,1326.00,0.08,0.08,0.09,0.07,...,24.99,23.41,158.80,1956.00,0.12,0.19,0.24,0.19,0.28,0.09
2,84300903,M,19.69,21.25,130.00,1203.00,0.11,0.16,0.20,0.13,...,23.57,25.53,152.50,1709.00,0.14,0.42,0.45,0.24,0.36,0.09
3,84348301,M,11.42,20.38,77.58,386.10,0.14,0.28,0.24,0.11,...,14.91,26.50,98.87,567.70,0.21,0.87,0.69,0.26,0.66,0.17
4,84358402,M,20.29,14.34,135.10,1297.00,0.10,0.13,0.20,0.10,...,22.54,16.67,152.20,1575.00,0.14,0.20,0.40,0.16,0.24,0.08


In [7]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
df['diagnosis']= le.fit_transform(df['diagnosis']) 
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.00,0.12,0.28,0.30,0.15,...,25.38,17.33,184.60,2019.00,0.16,0.67,0.71,0.27,0.46,0.12
1,842517,1,20.57,17.77,132.90,1326.00,0.08,0.08,0.09,0.07,...,24.99,23.41,158.80,1956.00,0.12,0.19,0.24,0.19,0.28,0.09
2,84300903,1,19.69,21.25,130.00,1203.00,0.11,0.16,0.20,0.13,...,23.57,25.53,152.50,1709.00,0.14,0.42,0.45,0.24,0.36,0.09
3,84348301,1,11.42,20.38,77.58,386.10,0.14,0.28,0.24,0.11,...,14.91,26.50,98.87,567.70,0.21,0.87,0.69,0.26,0.66,0.17
4,84358402,1,20.29,14.34,135.10,1297.00,0.10,0.13,0.20,0.10,...,22.54,16.67,152.20,1575.00,0.14,0.20,0.40,0.16,0.24,0.08


After using labelencoder on dignosis column the letter M which idicated malignant tumour is encoded with number 1 and benign tumour with 0.

In [9]:
#Splitting the dataset into X and y,X contains the first 10 columns which is used for training.
#y is the target column,which the model will predict
X=df.iloc[ :,1:10]
y=df.iloc[:,1]

In [10]:
X

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean
0,M,17.99,10.38,122.80,1001.00,0.12,0.28,0.30,0.15
1,M,20.57,17.77,132.90,1326.00,0.08,0.08,0.09,0.07
2,M,19.69,21.25,130.00,1203.00,0.11,0.16,0.20,0.13
3,M,11.42,20.38,77.58,386.10,0.14,0.28,0.24,0.11
4,M,20.29,14.34,135.10,1297.00,0.10,0.13,0.20,0.10
...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.00,0.11,0.12,0.24,0.14
565,M,20.13,28.25,131.20,1261.00,0.10,0.10,0.14,0.10
566,M,16.60,28.08,108.30,858.10,0.08,0.10,0.09,0.05
567,M,20.60,29.33,140.10,1265.00,0.12,0.28,0.35,0.15


In [11]:
y

0      M
1      M
2      M
3      M
4      M
      ..
564    M
565    M
566    M
567    M
568    B
Name: diagnosis, Length: 569, dtype: object

In [12]:
from azureml.train.automl import AutoMLConfig
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= X,
    label_column_name= 'diagnosis',
    n_cross_validations= 5)

In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_fba1e8bc-dc8d-4760-847e-5bac45aa1792

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

**************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
best_run, fitted_model = remote_run.get_output()
print('Best-run model: ',best_run)
print('\n fitted model: ',fitted_model)

Best-run model:  Run(Experiment: udacity-capstone,
Id: AutoML_fba1e8bc-dc8d-4760-847e-5bac45aa1792_61,
Type: None,
Status: Completed)

 fitted model:  Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                        coef0=0.0,
                                                                                        decision_function_shape='ovr',
                       

In [19]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_model,"best_model.pkl")

['best_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [20]:
script_file_name ='score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')
env=Environment.get(workspace=ws,name="AzureML-AutoML")

In [21]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=env)

In [22]:
from azureml.core.model import Model
import joblib
model = Model.register(workspace=ws, model_path="best_model.pkl", model_name="automl-test-model")

Registering model automl-test-model


In [23]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-Minimal")

In [24]:


aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'automl-cancer'
print(aci_service_name)

automl-cancer


In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice



service_name = 'custom-env-service'



service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[automl_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

In [32]:

print("Scoring uri:" , service.scoring_uri)

Scoring uri: http://ea0e0ff9-d344-4a54-884e-d6b576d36d61.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [39]:
scoring_uri = 'http://ea0e0ff9-d344-4a54-884e-d6b576d36d61.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

input_data = json.dumps({'data':[{
    								
    'radius_mean':17.99,
    'texture_mean':10.38,
    'perimeter_mean':122.80,	
    'area_mean':1001.00,	
    'smoothness_mean':0.12,	
    'compactness_mean':0.28,
    'concavity_mean':0.30,	
    'concave points_mean':0.15	
    }
    ]
        })
response = requests.post(scoring_uri, data=input_data, headers=headers)

print("Result:",response.text)

Result: "{\"result\": [\"M\"]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [34]:
print(service.get_logs())

2021-01-30T14:26:42,218472904+00:00 - iot-server/run 
2021-01-30T14:26:42,220589920+00:00 - rsyslog/run 
2021-01-30T14:26:42,220615920+00:00 - gunicorn/run 
2021-01-30T14:26:42,221779129+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [51]:
service.delete()